# Pipeline
파이프라인은 여러 변환 단계를 정확한 순서대로 실행할 수 있도록 하는 것이다. 

사이킷런은 연속된 변환을 순서대로 처리할 수 있도록 도와주는 Pipeline 클래스가 있다.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

### Define Pipeline

In [0]:
num_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy="median")),
                         ('std_scaler', StandardScaler())
])


[[10.]
 [20.]
 [30.]
 [40.]
 [50.]
 [nan]]
[[-1.54919334]
 [-0.77459667]
 [ 0.        ]
 [ 0.77459667]
 [ 1.54919334]
 [ 0.        ]]


In [0]:
# sample test data
test_data = np.array([10,20,30,40,50,np.nan]).reshape(-1,1)
print(test_data)

x = num_pipeline.fit_transform(test_data)

print(x)

### Pipeline Example

파이프라인 실습에 필요한 임의의 데이터를 생성한다.


In [0]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [0]:
# numpy array 형태로 변환. df.to_numpy()
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [0]:
# fit_transform 메소드로 파이프라인을 활용한 변환을 수행한다.
transformed = num_pipeline.fit_transform(df.values)
print(transformed)

[[-1.1769647  -1.1769647  -1.22474487 -1.22474487]
 [-0.09053575 -0.09053575  0.          1.22474487]
 [ 1.26750044  1.26750044  1.22474487  0.        ]]


In [0]:
# 개별적인 파이프라인의 단계를 확인할 수 있다.
num_pipeline.steps

[('imputer', SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                missing_values=nan, strategy='median', verbose=0)),
 ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]

In [0]:
num_pipeline.steps[0]

('imputer', SimpleImputer(add_indicator=False, copy=True, fill_value=None,
               missing_values=nan, strategy='median', verbose=0))

In [0]:
# 개별 변환단계의 일부 옵션 파라미터를 수정할 수 있다.
num_pipeline.set_params(imputer__strategy='constant')  # 결측치 대체 방법을 임의의 상수로 설정한다.
num_pipeline.set_params(imputer__fill_value=0.)

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=0.0,
                               missing_values=nan, strategy='constant',
                               verbose=0)),
                ('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)

In [0]:
transformed = num_pipeline.fit_transform(df.values)
print(transformed)

[[-1.1769647  -1.1769647  -0.39223227  0.        ]
 [-0.09053575 -0.09053575 -0.98058068  1.22474487]
 [ 1.26750044  1.26750044  1.37281295 -1.22474487]]


## iris 데이터셋을 활용한 실습
iris dataset을 표준화하고 로지스틱 회귀모형을 통해 분류 모형을 생성한다.
파이프라인을 사용하면 전처리에서 부터 모형 생성을 포함할 수 있다.

In [0]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

iris = datasets.load_iris()
features = iris.data
target = iris.target

In [0]:
pipe = Pipeline([
                 ("std_scaler", StandardScaler()),
                 ("classifier", LogisticRegression())
])

iris 데이터셋을 훈련데이터와 테스트데이터 셋으로 분리하고 fit 메소드를 사용하면 모델을 훈련할 수 있다.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size =0.2, stratify=target) 

model = pipe.fit(X_train, y_train)

훈련된 모형에 새로운 임의의 데이터를 적용하여 분류가 되는지 확인해 본다.

In [0]:
new_obs = [[0.5, 0.5, 0.5, 0.5]]
print(pipe.predict(new_obs))
print(pipe.predict_proba(new_obs))

[0]
[[9.81134179e-01 1.88593889e-02 6.43202405e-06]]
